In [4]:
class Location:
    def __init__(self, infection_rate):
        self.infection_rate = infection_rate

class Person:
    def __init__(self, location, infected=False):
        self.location = location
        self.recovery_prob = 0.8
        self.infected = infected

class Result:
    def __init__(self, day, location, num_susceptible, num_infected):
        self.day = day
        self.location = location
        self.num_susceptible = num_susceptible
        self.num_infected = num_infected

def simulate(people, locations, days):
    pass

# List of results -> Plot or list of plots
def plot_results(results):
    pass

In [5]:
locations = [Location(0.1)]
people = [Person(locations[0], True), Person(locations[0])]
results = simulate(people, locations, 10)
plot_results(results)